# Tutorat M1 S2 Informatique DS4H

### Subject:

#### Re-structuration intelligente d'un jeu de donnée de debats politiques pour l'extraction de structures argumentaires

L'objectif du travail à réaliser est de structurer des données nécessaires à l'étude des composantes et des relations au sein des débats politiques qui ont eu lieu lors des élections du président des États-Unis de 1960 à 2016. Les débats se présentent sous la forme d'un dialogue entre un candidat et l'autre, qui répondent aux questions posées par un orateur sur divers sujets tels que l'économie, la sécurité, l'éducation, la guerre, les soins de santé, etc. Chaque débat a été divisé en sections en tenant compte des différents sujets abordés.

Tous les débats ont été annotés d'un point de vue argumentatif. Des annotations concernant les composantes argumentatives sont présentes à l'intérieur :
- Conclusions
- Prémisses
et des annotations faisant référence aux relations entre ces deux composants :
- Attaque
- Soutien
- Équivalent

L'objectif du projet est de concevoir et implémenter une structure de données textuels qui soit facile à manipuler pour la réalisation d'une des nombreuses tâches du TAL, à savoir l'extraction d'arguments.
Plus précisément, il s’agit de deux structures :
1) Un jeu de données référençant les composants (Claim, Premise) représentés par les colonnes suivantes :
- Ligne de dialogue
- Composants de l'argumentation
- Schéma BIO des composants
- Caractéristiques linguistiques, lexicales, grammaticales, syntaxiques, etc... (Chaque caractéristique séparée par une colonne) concernant le composant considéré
2) Un ensemble de données se référant aux relations (Attaque, Soutien, Équivalent) regroupées par section et représentées par les colonnes suivantes :
- Composante 1 (Claim/Premise)
- Composante 2 (Claim/Premise)
- Type de relation (Attaque/Soutien/Équivalent)
- Schéma BIO des composants et des relations avec leur distance
- Caractéristiques linguistiques, lexicales, grammaticales, syntaxiques, etc. (chaque caractéristique séparée par une colonne) concernant la relation considérée

## Questions
* Do we want to remove the Speaker Name from the sentences ? (In the contexte column)
* What do we keep from the components csv columns ?
* For the context 1, what if there are only 1 sentence in the component ? how can we get the 3 closest sentences ?

In [150]:
# import the libraries
import pandas as pd
from nltk.tokenize import sent_tokenize

In [151]:
# open the data file
components_data = pd.read_csv('./data/test_components.csv')
speeches_data = pd.read_csv('./data/test_speeches.csv')
# work with only the first 5 rows
# components_data = components_data.head(5)

In [152]:
# create new dataframe that copy the components_data but without the "Previous_Sentence" and "Next_Sentence" columns
components_data_context = components_data.drop(['Current_Sentence', 'Previous_Sentence', 'Next_Sentence'], axis=1)
# add columns for contexts
components_data_context['Context1'] = ''
components_data_context['Context2'] = ''

In [153]:
# for each component, find the speeches that mention it
for index, row in components_data.iterrows():
    textToFind = row.Text
    # find the speeches that have the component in the text
    speeches = speeches_data[speeches_data['Speech'].str.find(textToFind) != -1]
    
    # make a try catch block to handle the case where the component is not found in the speeches
    try:
        # tokenize the speech into sentences
        sentences = sent_tokenize(speeches['Speech'].values[0])
        
        context1 = ''
        # get the sentence that contains the component
        for index, sentence in zip(range(0,len(sentences)), sentences):
            if sentence.find(textToFind) != -1:
                if(index > 0):
                    context1 = sentences[index-1]
                # add the current sentence
                context1 = context1 + ' ' + sentence
                if(index < len(sentences)-1):
                    context1 = context1 + ' ' + sentences[index+1]
                break
        # add context1
        components_data_context.loc[index, 'Context1'] = context1
        # add full speech to the context2 column
        components_data_context.loc[index, 'Context2'] = speeches['Speech'].values[0]
    except:
        print('Component not found in speeches: ' + textToFind+ ' ' + str(index))
        
    

Component not found in speeches: I believe it my responsibility as the leader of the Democratic party in 1960 to try to warn the American people that in this crucial time we can no longer afford to stand still We can no longer afford to be second best 56
Component not found in speeches: Anybody that says America has been standing still for the last seven and a half years hasn't been traveling in America He's been in some other country 75
Component not found in speeches: There were eleven dictators when we came into power in 1953 in Latin America There are only three left 94
Component not found in speeches: Let's look at Africa Twenty new countries in Africa during the course of this Administration Not one of them selected a Communist government 95
Component not found in speeches: the tax program of the Ford administration will bring jobs where people are, and help to revitalize those cities as they can be 603
Component not found in speeches: I do make a pledge that in the next ten days